In [1]:
import pandas as pd
import json

In [2]:
with open('output_steam_games.json', 'r', encoding='utf-8') as archivo:
    data = [json.loads(line) for line in archivo]

data_games = pd.DataFrame(data)
data_games.drop(data_games.index[0:88310], inplace=True)

In [3]:
# Eliminar aquellas columnas que no serán relevantes para el análisis futuro.
data_games.drop(['app_name','early_access','url','reviews_url','tags','publisher','specs'], axis=1, inplace = True)
data_games.reindex()

# Extraer únicamente el año de la columna 'release_date'.
data_games['release_date'] = pd.to_datetime(data_games['release_date'], errors='coerce').dt.year

# Eliminar los valores nulos en cada columna.
data_games = data_games.dropna(subset=['genres'])
data_games = data_games.dropna(subset=['title'])
data_games = data_games.dropna(subset=['release_date'])
data_games = data_games.dropna(subset=['price'])
data_games = data_games.dropna(subset=['id'])
data_games = data_games.dropna(subset=['developer'])

# Modificar el tipo de dato de la columna 'release_date'.
data_games['release_date'] = data_games['release_date'].astype(int)
data_games['release_date'] = data_games['release_date'].astype(str)

#  Modificar los valores de la columna 'price' para asignar el número '0' a aquellos casos donde se indique que el producto es gratuito.
data_games['price'] = data_games['price'].replace({'Free To Play': 0})
data_games['price'] = data_games['price'].replace({'Free to Play': 0})
data_games['price'] = data_games['price'].replace({'Play for Free!': 0})

# Convertir todos los valores a numéricos y eliminar aquellos que contengan texto
data_games = data_games[pd.to_numeric(data_games['price'], errors='coerce').notnull()]

# Renombrar la columna 'id' como 'item_id'.
data_games = data_games.rename(columns={'id':'item_id'})

# Separar la columna 'genres' en varias columnas en un nuevo DataFrame y luego concatenarlo con el DataFrame origina
data_games1 = data_games['genres'].apply(pd.Series)
data_games2 = pd.concat([data_games, data_games1], axis=1)

# Eliminar la columna 'genres' junto con las columnas generadas al separarla en varias columnas, conservando solo la primera columna ['0'] como el único género del juego.
data_games2.drop(['genres'], axis=1, inplace = True)
data_games2.drop([1,2,3,4,5,6,7,8,9], axis=1, inplace = True)

# Seleccionar las variables con las características más importantes según criterio establecido.
data_games2 = data_games2[['item_id','title',0,'developer','release_date','price']]
data_games2 = data_games2.rename(columns={0: 'genre'})  

# Eliminar duplicados en caso de que existan.
data_games2 = data_games2.drop_duplicates()

# Exportar la tabla como un archivo separado por comas con el nombre 'games'.
data_games2.to_csv('games.csv', index=False)